![Piggy bank](piggy_bank.jpg)

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to `boolean` data type:<br> `1` if `"yes"`, otherwise `0` |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0` |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type:<br> `1` if `"success"`, otherwise `0`. |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0`. |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
"""Convert all columns for requisite necessary
"""

columns_dtypes = {
    "client_id": int,
    "age": int,
    "number_contacts": int,
    "contact_duration": int,
    "previous_campaign_contacts": int,
    "job": object,
    "martial": object,
    "education": object,
    "last_contact_date": str,
    "cons_price_idx": float,
    "euribor_three_months": float,
    "credit_default": object,
    "mortgage": object,
    "previous_outcome": object,
    "campaign_outcome": object,
}


df = pd.read_csv("bank_marketing.csv", dtype=columns_dtypes)

In [ ]:
"""Mapping all months from numbers, and concatenate columns for create a new column
"""

month_map = {
    "jan": 1,
    "feb": 2,
    "mar": 3,
    "may": 5,
    "apr": 4,
    "jun": 6,
    "jul": 7,
    "aug": 8,
    "sep": 9,
    "oct": 10,
    "nov": 11,
    "dez": 12,
}
df["month"] = df["month"].map(month_map)
df["year"] = 2022
df["last_contact_date"] = pd.to_datetime(df[["year", "month", "day"]])

In [ ]:
""" Function for replace letters. 
Args: Column Name, Letter 1 and Letter 2, 
if NaN is True remove str and using replace."""


def replace_letter(column_name, letter1, letter2, NaN=False):

    if not NaN:
        df[column_name] = df[column_name].str.replace(letter1, letter2)

    elif NaN:
        df[column_name] = df[column_name].replace(letter1, letter2)

    return df


df = replace_letter("job", ".", "_")
df = replace_letter("education", ".", "_")
df = replace_letter("education", "unknown", np.NaN, NaN=True)

In [ ]:
"""Function for mapping and replace values for bool values
    Args: Column Name, Mapping Dict for all values are is necessary to replace a bool value
"""


def convert_values_to_boolean(column_name, mapping_dct):
    df[column_name] = df[column_name].map(mapping_dct)
    return df


values_mapping = {
    "yes": 1,
    "no": 0,
    "unknown": 0,
    "success": 1,
    "nonexistent": 0,
    "failure": 0,
}

df = convert_values_to_boolean("credit_default", values_mapping)
df = convert_values_to_boolean("mortgage", values_mapping)
df = convert_values_to_boolean("previous_outcome", values_mapping)
df = convert_values_to_boolean("campaign_outcome", values_mapping)

In [ ]:
"""Function for define type for column
    Args: Column Name and type value (astype)
"""


def convert_column_to_type_value(column_name, type_value):
    df[column_name].astype(type_value)


df["credit_default"] = convert_column_to_type_value("credit_default", bool)
df["mortgage"] = convert_column_to_type_value("mortgage", bool)
df["previous_outcome"] = convert_column_to_type_value("previous_outcome", bool)
df["campaign_outcome"] = convert_column_to_type_value("campaign_outcome", bool)

In [ ]:
"""Selecting all columns necessary for create a csv file
"""

column_client = [
    "client_id",
    "age",
    "job",
    "marital",
    "education",
    "credit_default",
    "mortgage",
]
df_client = df[column_client]

df_client.to_csv("client.csv", index=False)

column_campaign = [
    "client_id",
    "number_contacts",
    "contact_duration",
    "previous_campaign_contacts",
    "previous_outcome",
    "campaign_outcome",
    "last_contact_date",
]
df_campaign = df[column_campaign]
df_campaign.to_csv("campaign.csv", index=False)


column_economics = ["client_id", "cons_price_idx", "euribor_three_months"]
df_economics = df[column_economics]
df_economics.to_csv("economics.csv", index=False)

In [ ]:
df = pd.read_csv("bank_marketing.csv")

for col in ["credit_default", "mortgage", "previous_outcome", "campaign_outcome"]:
    print(col)
    print("--------------")
    print(df[col].value_counts())